# Solution 4 - Principal Component Analysis
## Data Analysis
### FINM August Review 

Mark Hendricks

hendricks@uchicago.edu

## 1 Principal Components

Use the single-name equity data from the data set mentioned above. (That is, leave out SPY and SHV.)

1. Calculate the principal components of the return series.


2. Report the eigenvalues associated with these principal components. Report each eigenvalue as a percentage of the sum of all the eigenvalues. This is the total variation each PCA explains.


3. How many PCs are needed to explain 75% of the variation?


4. Calculate the correlation between the first (largest eigenvalue) principal component with each of the 22 single-name equities. Which correlation is highest?


5. Calculate the correlation between the SPY and the first, second, third principal components.

# <span style='color:red'>Solution 1</span>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")

In [2]:
FILEIN = '../data/spx_weekly_returns_single_names.xlsx'
rets = pd.read_excel(FILEIN, sheet_name=2, index_col=0)
spy = pd.read_excel(FILEIN, sheet_name=3, index_col=0)[['SPY']]

NSECS = rets.shape[1]

In [3]:
# data = pd.read_excel("single_name_return_data.xlsx", sheet_name="total returns").set_index("Date")
# equities = data.drop(columns=['SPY', 'SHV'])
# spy = data[['SPY']]
# rf = data[['SHV']]

##### 1.

In [4]:
# Steps to calculate PC of the return series:

pca = PCA(svd_solver='full')
pca.fit(rets.values)

pca_factors = pd.DataFrame(pca.transform(rets.values), 
                           columns=['PC {}'.format(i+1) for i in range(pca.n_components_)], 
                           index = pd.to_datetime(rets.index))
display(pca_factors.tail().style.format('{:,.4f}'))

##### 2.

In [5]:
explained_var = pd.DataFrame(data = pca.explained_variance_ratio_,
                                 index = pca_factors.columns, 
                                 columns = ['Explained Variance'])
explained_var['Cumulative Explained Variance'] = explained_var['Explained Variance'].cumsum()

display(explained_var.style.format('{:,.2%}'))

,Explained Variance,Cumulative Explained Variance
PC 1,38.91%,38.91%
PC 2,5.67%,44.58%
PC 3,3.78%,48.36%
PC 4,2.77%,51.14%
PC 5,2.07%,53.21%
PC 6,1.49%,54.70%
PC 7,1.25%,55.95%
PC 8,1.16%,57.11%
PC 9,1.05%,58.16%
PC 10,1.03%,59.19%


##### 3.

Looking at the cumulative percentage of explained variance, the first 5 Principal Component Factors are enough to explain over 55% of the variation in the single name equities.

Just the first PC explains over 40%!

###### 4.

In [6]:
rets_corr = rets.copy()
rets_corr.insert(0, 'PC 1', pca_factors['PC 1'])

corr_table = pd.DataFrame(rets_corr.corr().iloc[0, 1:]).sort_values('PC 1').style.format('{:,.2%}')
display(corr_table)

,PC 1
CPB,11.36%
KR,13.84%
HRL,15.32%
GIS,16.28%
SJM,17.79%
CLX,17.98%
CHD,21.15%
NEM,23.60%
GILD,26.79%
EQT,27.73%


##### 5. 

In [7]:
factors = pca_factors[['PC 1', 'PC 2', 'PC 3']]
factors['SPY'] = spy
print('Correlation with SPY:')
display(factors.corr().iloc[3, :3])

Correlation with SPY:


PC 1    0.941321
PC 2   -0.223747
PC 3    0.060345
Name: SPY, dtype: float64

***

## 2 PCR and PLS

1. Principal Component Regression (PCR) refers to using PCA for dimension reduction, and then utilizing the principal components in a regression. Try this by regressing SPY on the first 3 PCs calculated in the previous section. Report the r-squared.


2. Calculate the Partial Least Squares estimation of SPY on the 22 single-name equities. Model it for 3 factors. Report the r-squared.


3. Compare the results between these two approaches and against penalized regression seen in the past homework.

# <span style='color:red'>Solution 2</span>

###### 1.

In [8]:
X_PCR = factors[['PC 1', 'PC 2', 'PC 3']]

model_PCR = LinearRegression().fit(X_PCR, spy)
print('PCR R-squared: ' + str(round(model_PCR.score(X_PCR, spy),3)))

PCR R-squared: 0.94


###### 2.

In [9]:
X_PLS = rets
y_PLS = spy

model_PLS = PLSRegression(n_components=3).fit(X_PLS, y_PLS)

print('PLS R-squared: ' + str(round(model_PLS.score(X_PLS, y_PLS),3)))

PLS R-squared: 0.983


##### 3. 

PCR and PLS both seek to maximize the ability to explain the variation in y variable, and therefore they will have high $R^2$ in-sample. When using LASSO or Ridge as our model, we are conservatively forming factors, and penalizing for additional factors. This makes in-sample $R^2$ lower as we saw in Homework #5, but may make more robust OOS predictions.

***

### <span style='color:red'>Footnote</span>

For those intrested in the implementation of PCA, here is a simple code in Python:

In [10]:
# Steps to calculate PC of the return series:

### 1. Center the return series (mean = 0)

rets_centered = rets - np.mean(rets, axis = 0)

### 2. Calculate the Covariance Matrix

cov = np.cov(rets_centered , rowvar = False)

### 3. Calculating Eigenvalues and Eigenvectors of the covariance matrix

eigen_values , eigen_vectors = np.linalg.eigh(cov)

### 4. Sort the eigenvalues and corresponding eigenvectors in descending order

sorted_index = np.argsort(eigen_values)[::-1]
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:,sorted_index]

### 5. Transform the Data

PCA_factors = np.dot(sorted_eigenvectors.transpose(),rets_centered.transpose()).transpose()
display(pd.DataFrame(PCA_factors))

,0,1,2,3,4,5,6,7,8,9,...,436,437,438,439,440,441,442,443,444,445
0,-0.703385,0.077855,0.239065,-0.172304,-0.140798,-0.211526,-0.001276,-0.027892,-0.096028,-0.009757,...,0.000278,0.000039,1.007017e-15,3.053127e-15,4.701299e-15,-1.554825e-15,5.650144e-16,1.246768e-15,2.401134e-15,-1.518271e-15
1,0.227620,0.064802,0.005063,0.178815,0.182955,-0.054410,0.034111,0.027683,-0.041873,0.013611,...,0.000120,-0.000289,-5.319948e-15,-3.844630e-16,8.478584e-16,-3.797561e-16,2.518100e-15,-1.766414e-15,1.683057e-15,1.573819e-15
2,0.343365,-0.066115,0.080619,0.204560,-0.001394,-0.133521,0.269311,-0.055258,-0.184666,-0.083771,...,-0.000126,-0.000106,-1.915046e-15,5.397948e-17,-4.195032e-15,1.564001e-15,3.351239e-16,1.885581e-15,3.459668e-15,3.520829e-16
3,-0.639496,-0.166702,0.215107,-0.061996,-0.165133,0.033232,0.429579,-0.002246,-0.057776,0.033772,...,-0.000007,0.000235,-5.507973e-16,5.528095e-15,-5.621492e-15,2.990884e-15,3.018617e-15,-4.315505e-15,1.426181e-15,1.232534e-16
4,-0.431774,0.114435,0.002420,-0.046339,-0.239280,-0.073124,-0.191123,-0.147314,-0.183536,-0.170020,...,-0.000258,-0.000073,-2.912816e-15,-1.155535e-15,-2.280439e-15,1.875856e-15,7.935956e-16,-3.260806e-15,-3.580223e-16,5.488189e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,0.322281,0.016100,0.138986,0.081755,-0.176777,-0.062232,0.079106,-0.001161,-0.015817,0.025151,...,0.000114,0.000267,4.200152e-15,6.198497e-15,-8.164267e-16,3.072799e-15,6.604427e-15,-2.939906e-15,-1.219816e-15,-1.182815e-15
435,0.170134,0.086424,-0.002675,0.021857,0.086797,-0.022459,0.001667,0.105334,0.111094,-0.027112,...,0.000103,-0.000238,-7.932358e-16,2.018285e-15,-3.105267e-15,-1.331176e-15,7.382263e-15,-8.745626e-16,2.931768e-15,-5.182197e-15
436,-0.333449,0.044161,-0.266674,-0.113836,0.145057,0.063181,0.069739,-0.111839,0.112183,-0.169284,...,0.000123,-0.000082,-2.279480e-15,-1.088826e-15,-7.548321e-15,3.295033e-15,1.855354e-15,-4.185049e-15,4.207163e-15,3.620195e-15
437,-0.044586,-0.207634,0.204942,0.006273,0.031917,0.045634,0.061622,0.016448,-0.032551,-0.031903,...,0.000348,0.000325,6.635098e-15,6.039213e-16,1.577186e-14,-3.786253e-15,3.453634e-15,3.592830e-15,3.489523e-16,-7.168631e-16
